# Plot SFINCS Results

This notebook post-processes the results from the SFINCS model into maps of maximum water depth in flooded areas (masking permanent water bodies provided a mask is available). Outputs are 
1. Timeseries at observation stations compared against measurements.
2. Flood map with maximum water depth and extent for the event duration.
3. Animated map with time slider. Permanent water bodies not masked yet!
4. Video animation.  

Results are all saved to the simulation folder

In [35]:
from hydromt_sfincs import SfincsModel
import hydromt_sfincs.utils as utils
import numpy as np
from pathlib import Path 
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.colors import ListedColormap
import pandas as pd
import geopandas as gpd
import hvplot.xarray
import xarray as xr
from pathlib import Path


## USER INPUT

Set the model directory.
Specify the DEM file to produce downscaled water depth maps.  
Provide filename for the output map. 
Water level measurements to compare against model results (received from a local tide gauge). TO DO: replace with automatic download.

In [36]:
# simulation folder
# main_dir = Path(r"p:\11209308-floodadapt-trinidad\InterTwin")
main_dir = Path(r"c:\Users\winter_ga\Offline_data\projects\InterTwin")
simulation_path = main_dir.joinpath("model\model_20m_nowind_dikes_overtopping_0-8")
# DEM file for downscaling
demfile = main_dir.joinpath("Geodaten\DEM.tif")
# output filename for flood map
floodmap_fn = "max_waterdepth.tif"
# mask for permanent water bodies
land_mask = main_dir.joinpath("Geodaten\landmask.gpkg")

# water level observations
# offset of tide gauge from local datum
waterlevel_offset = -498
wl_csv = []
wl_csv.append(main_dir.joinpath("Wasserstaende\pegelonline-barhft-W-20231015-20231023.csv"))
wl_csv.append(main_dir.joinpath("Wasserstaende\pegelonline-barth-W-20231015-20231023.csv"))
wl_csv.append(main_dir.joinpath("Wasserstaende\pegelonline-althagen-W-20231015-20231023.csv"))


# Read model results

In [37]:
# read results
model = SfincsModel(root=simulation_path, mode="r+")
model.read()

model.read_results()

# save station data as csv
df = pd.DataFrame(index=model.results["point_zs"].time.values)
if len(model.results["point_zs"].stations) > 0:
    for ii in np.arange(len(model.results["point_zs"].stations)):
        model.results["point_zs"].isel(stations=ii).to_dataframe().to_csv(
            Path(simulation_path).joinpath(f"station_data_{ii:03d}.csv")
        )
        df[ii] = model.results["point_zs"].isel(stations=ii).values



ValueError: Failed to decode variable 'timemax': unable to decode time units 'seconds since 2023-10-15 07:00:00' with 'the default calendar'. Try opening your dataset with decode_times=False or installing cftime if it is not installed.

# Import observed water levels 

In [ ]:
# read observations
wl_df = []
for ii,fn in enumerate(wl_csv):
    wl_df.append(pd.read_csv(fn,header=0, delimiter=";"))
    wl_df[ii]["timestamp"] = pd.to_datetime(wl_df[ii]["timestamp"])
    wl_df[ii] = wl_df[ii].set_index("timestamp")
    if ii>=1:
        wl_df[0] = wl_df[0].join(wl_df[ii],rsuffix=str(ii))

obs_df = (wl_df[0] + waterlevel_offset)/100
obs_df = obs_df.rename(columns={"value":"Barhoeft","value1":"Barth","value2":"Althagen"})

obs_df


# Plot comparison of observed and modelled water levels

In [ ]:

fig, ax = plt.subplots()
df.plot(ax=ax,y=[0,1,2])
clr=[]
print(ax.get_lines())
for n,li in enumerate(ax.get_lines()):
    clr.append(li.get_color())
obs_df.plot(ax=ax, style="--",cmap=ListedColormap(clr))
plt.grid()
# ax.set_ylim(bottom=0)
labels = [str(station).replace(" ", "").replace("b","") for station in model.results["point_zs"].station_name.values]
plt.legend(obs_df.columns.to_list())
ax.set_ylabel("Waterlevel [m above NHN]")

plt.savefig(Path(simulation_path).joinpath("station_data.png"))




# Generate downscaled flood map 
This shows the  maximum water depths over the event duration. Use mask if available to mask permanent water bodies.

In [ ]:
# read land mask
gdf_mask = gpd.read_file(land_mask)
# read dem file for high resolution flood depth map
dem = model.data_catalog.get_rasterdataset(demfile)

zsmax = model.results["zsmax"].max(dim="timemax")
zsmax.attrs["units"] = "m"
# writing the geotiff to the scenario results folder, masking permanent water bodies
utils.downscale_floodmap(
    zsmax=zsmax,
    dep=dem,
    hmin=0.01,
    floodmap_fn=str(Path(simulation_path).joinpath(floodmap_fn)),
    gdf_mask = gdf_mask
)



Create timeslider of flood water depth (It's very slow!!!)

In [ ]:

# hmin = 0.05
# da_h = model.results["h"].copy()
# da_h = da_h.where(da_h > hmin).drop("spatial_ref")
# da_h.attrs.update(long_name="flood depth", unit="m")

# # # mask permanent water bodies -> not working yet
# # inifile = r"c:\Users\winter_ga\Offline_data\projects\InterTwin\Geodaten\zsini_regridded_20m_final.tif"
# # ini_ds = xr.open_dataset(inifile, engine="rasterio")
# # ini_ds["zsini"] = ini_ds["band_data"]
# # ini_ds = ini_ds.drop(["band_data","band"])
# # id=ini_ds.sel(band=0).where(0,np.nan)

# da_h.hvplot.image(
#     x="x",
#     y="y",
#     groupby="time",
#     tiles=True,
#     cmap="Reds",
#     frame_width=400,
#     crs=25833
# )

Create video animation of flooding. Requires the installation of FFPMG, so I commented this out. This takes a while...

In [ ]:
# # create animation
# hmin = 0.05
# da_h = model.results["h"].copy()
# da_h = da_h.where(da_h > hmin).drop("spatial_ref")
# da_h.attrs.update(long_name="flood depth", unit="m")
# # create hmax plot and save to mod.root/figs/sfincs_h.mp4
# # requires ffmpeg install with "conda install ffmpeg -c conda-forge"

# step = 1  # one frame every <step> dtout
# cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}


# def update_plot(i, da_h, cax_h):
#     da_hi = da_h.isel(time=i)
#     t = da_hi.time.dt.strftime("%d-%B-%Y %H:%M:%S").item()
#     ax.set_title(f"SFINCS water depth {t}")
#     cax_h.set_array(da_hi.values.ravel())


# fig, ax = model.plot_basemap(
#     fn_out=None, variable="", bmap="sat", plot_bounds=False, figsize=(11, 7)
# )
# cax_h = da_h.isel(time=0).plot(
#     x="x", y="y", ax=ax, vmin=0, vmax=3, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
# plt.close()  # to prevent double plot

# ani = animation.FuncAnimation(
#     fig,
#     update_plot,
#     frames=np.arange(0, da_h.time.size, step),
#     interval=1000,  # ms between frames
#     fargs=(
#         da_h,
#         cax_h,
#     ),
# )

# # to show in notebook:
# from IPython.display import HTML

# HTML(ani.to_html5_video())

# # to save to mp4
# ani.save(str(Path(simulation_path).joinpath('sfincs_h.mp4')), fps=4, dpi=200)

In [ ]:
# close model results
del model